# 개선해야하는 부분

    1) 유사도 0.6 이상 조건 넣기
    2) [사진,포토] 등 불필요한 단어 제거
    3) 불용어처리 가능하게 하기 

# 제목유사도 측정

In [ ]:
import re
import pandas as pd
import numpy as np
import json
#from bs4 import Beautiful Soup
# from nltk.corpus import Beautiful Soup
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
import konlp
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
#헤더추가 (title,content,time)
all =pd.read_csv('dataset3000.csv',names=['title','content','time','nan'])
all.head()
all.shape
all.dtypes

title       object
content     object
time        object
nan        float64
dtype: object

In [6]:
all.head()

title  \
0                 '[사진]조여정,'우아한 플라워원피스 입고''   
1              '[프로필]국세청 과장급 전보자(7월 22일 자)'   
2           '[조이HD]춘사영화제 참석한 봉준호 감독-한진원 작가'   
3                 '[사진]클라라,'시선 올킬 블루 롱드레스''   
4  '[사진]7개부문 노미네이트 기생충 봉준호 감독,'엄청난 환호 받으며''   

                                             content              time  nan  
0  '제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...  2019-07-20 16:40  NaN  
1  '조사기획1계장 ▲경주세무서장 ▲국세청 인도네시아 주재관 ▲외교부 파견 ▲서울지방국...  2019-07-20 16:40  NaN  
2  '말레이시아 Finas(말레시아 영화진흥위)의 신임 대표인 Ahmad Idham을 ...  2019-07-20 16:40  NaN  
3  '제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...  2019-07-20 16:40  NaN  
4  '제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...  2019-07-20 16:40  NaN

# trian 과 test 나누기 

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
train, test = train_test_split(all, test_size=0.2)

In [9]:
train.info

<bound method DataFrame.info of                                                   title  \
378                           '핀란드 이어 인도네시아도 부산발 직항 뜰까'   
2295                                     '수마트라 섬 갔던 제비'   
2570                    '블록체인 소셜 앱에 빠진 '인니' 국내사 진출 이어져'   
2776                              '옥천 복숭아 홍콩 시장 공략 나선다'   
287             '여자친구, 아시아 투어 싱가포르 공연 성료!! 아시아 투어 순항 中'   
11                           '[조이HD]'춘사영화제' 문지인, 민트여신~'   
790                          '[대일논단] 동인도회사의 기업가 정신과 창업'   
2837                 '아시아 유통기업 '이커머스' 약진…알리바바 1위·롯데 8위'   
1627               'ACC브런치콘서트 창작판소리 '스토리 인 아시아' 31일 공연'   
2284  '[글로벌-Biz 24] 시설안전공단, 중소기업과 손잡고 인도네시아 특수교량관리시...'   
744                         '발리, 녹색부담금 도입 등 폐기물 관리 본격화'   
462                                    '인도네시아 서브를 보여주마'   
603                   '매일 잠든 2세 아들 입을 테이프로 막는 여성, 이유는?'   
1032                '\선거포스터 포샵은 사기\"vs\"단점 보완도 못하나?\"'"   
2531                              '암호화폐 커뮤니티 코인빽 웹툰 런칭'   
1194                    

In [10]:
test.info

<bound method DataFrame.info of                                                   title  \
1767                        '‘유니콘 4社 탄생지’ 인도네시아가 핫한 이유'   
1856                   '7월 25일 외국환시세(KEB하나은행 1차 고시 기준)'   
227                        '[MD포토] 물오른 미모 수영 '바람은 거들뿐''   
1213                               '아메리칸항공 737맥스 금지 연기'   
2868        '[포토] ABCC FINTECH INDONESIA, 양해각서(MOU) 체결'   
2208  '[공식] NCT DREAM, 신곡 'We Boom' 아이튠즈 전 세계 21개 지역...   
531             'SM그룹 KLCSM, 선박 안전운항을 위한 컨퍼런스 및 세미나 개최'   
257                         '소녀시대 수영 ‘분위기가 다했네’[포토엔HD]'   
833             '한국-인도네시아 정보보안 스타트업 교류 확대..KISA, 행사 개최'   
952                     '합천군청, 「합천군, 냉동 딸기 인도네시아 첫 수출」'   
2809                  ''팔아야 귀국' 시간이 흘러도 손님들은 줄지 않았다...'   
1804                            '인도네시아 빈탄섬 해변 언덕 위 풀빌라'   
2241                      '‘제시카’ 이번 화보에서 제시카는 한 편의 ...'   
2198  '[팔아야 귀국] 김동현, 장동민 구박에 \제 가방 어디 있어요? 그냥 집에 갈래....   
2853      ''네마프 2019', 롯데시네마 홍대입구서 '덴마크 비디오아트 특별전' 개최!'   
788                     

In [11]:
train_title=list(train['title'])

In [16]:
for i in range(len(train_title)) :
    train_title[i] = re.sub("[^가-힣ㄱ-ㅎ ㅏ-ㅣ\\s]","", train_title[i])

print(train_title)

['핀란드 이어 인도네시아도 부산발 직항 뜰까', '수마트라 섬 갔던 제비', '블록체인 소셜 앱에 빠진 인니 국내사 진출 이어져', '옥천 복숭아 홍콩 시장 공략 나선다', '여자친구 아시아 투어 싱가포르 공연 성료 아시아 투어 순항 ', '조이춘사영화제 문지인 민트여신', '대일논단 동인도회사의 기업가 정신과 창업', '아시아 유통기업 이커머스 약진알리바바 위롯데 위', '브런치콘서트 창작판소리 스토리 인 아시아 일 공연', '글로벌  시설안전공단 중소기업과 손잡고 인도네시아 특수교량관리시', '발리 녹색부담금 도입 등 폐기물 관리 본격화', '인도네시아 서브를 보여주마', '매일 잠든 세 아들 입을 테이프로 막는 여성 이유는', '선거포스터 포샵은 사기단점 보완도 못하나', '암호화폐 커뮤니티 코인빽 웹툰 런칭', '시노펙스 인니 수방군에 정수처리시설 공급', '강원랜드  하이원 중고등학생 드림 원정대 발대식 개최', '카페베네의 두 번째 심기일전 기대 혹은 우려', '속보 인도네시아 발리 인근 해상에서 규모 지진쓰나미 경보는', '최강 듀오 동방신기 홍콩 앵콜콘 퍼펙트한 마무리', '텐센트 체스러쉬 글로벌 스타 챌린지 개최', '경북 중소기업 수출 지원 러시아인니에 교두보', ' 인니산 스테인리스강 반덤핑 관세 부과', '경남과학교육원 국내 첫 제비 이동경로 밝혔다', '앱애니 분기 모바일 게임 지출액 억 달러전체의 ', '국표원 인도인도네시아에 무역기술장벽 해소 요청', '인도네시아 몰루카제도 규모  지진으로 가옥 파손', '크래프톤  아크시스템웍스와 미스트오버 아시아 서비스 계약', '뚜두뚜두 블랙핑크는 지난 월 일 현지시간', '농구 국가대표팀 존스컵에서 이란 꺾고 연승 행진', '포토 수영 꾸민듯 안꾸민듯', '아시아 최대 패션문화마켓 패션코드   참가사 모집', '신간안내', '팔아야귀국 신봉선 팝업스토어 장식한 사진에 셀프디스 옛날 코라서', '부산시 인도네시아 산업기계 전시회 참가업체 모집', '에이스 자카르타 푸드 페어 참석현지 팬들과 소

# train title 문장의 명사만 추가한 리스트 만들기

### 여기선 TF-idf 사용

참고사이트: https://code-ing.tistory.com/2?category=728528

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

okt=Okt()

doc_nouns_list=[]

for doc in train_title:
    nouns=okt.nouns(doc)
    doc_nouns=''
    
    for noun in nouns:
        doc_nouns+=noun + ' '
    doc_nouns_list.append(doc_nouns)
    
for i in range(0,len(train_title)):
    print('doc' +str(i+1)+ ' : ' +str(doc_nouns_list[i]))

doc1 : 핀란드 인도네시아 부산 발 직항 
doc2 : 수마트라 섬 제비 
doc3 : 블록 체인 소셜 앱 인니 국내 사 진출 
doc4 : 옥천 복숭아 홍콩 시장 공략 
doc5 : 여자친구 아시아 투어 싱가포르 공연 료 아시아 투어 순항 
doc6 : 이춘사 영화제 문지 민트 여신 
doc7 : 대일 논단 동인도회사 기업가 정신과 창업 
doc8 : 아시아 유통 기업 이커머스 약진 알리바바 위 롯데 위 
doc9 : 브런치 콘서트 창작 판소리 스토리 인 아시아 일 공연 
doc10 : 글로벌 시설 안전 공단 중소기업 인도네시아 특수 교량 관리 시 
doc11 : 발리 녹색 부담 금 도입 등 폐기물 관리 본격 
doc12 : 인도네시아 서브 
doc13 : 매일 세 아들 입 테이프 여성 이유 
doc14 : 선거 포스터 포샵 사기 단점 보완 못 하나 
doc15 : 암호 화폐 커뮤니티 코인 빽 웹툰 런칭 
doc16 : 시노펙 스 인니 방 정수 처리 시설 공급 
doc17 : 강원랜드 하이 중 고등학생 드림 원정 발대식 개최 
doc18 : 카페베네 두 심기 일전 기대 우려 
doc19 : 속보 인도네시아 발리 인근 해상 규모 지진 쓰나미 경보 
doc20 : 최강 듀오 동방신기 홍콩 앵콜 콘 퍼펙트 마무리 
doc21 : 텐센트 체스 러쉬 글로벌 스타 린지 개최 
doc22 : 경북 중소기업 수출 지원 러시아 인니 교두보 
doc23 : 인니 산 스테인리스강 반덤핑 관세 부과 
doc24 : 경남 과학 교육 국내 첫 제비 이동 경로 
doc25 : 앱 애니 분기 모바일 게임 지출 액 억 달러 전체 
doc26 : 국표 인도 인도네시아 무역 기술 장벽 해소 요청 
doc27 : 인도네시아 루카 제도 규모 지진 가옥 파손 
doc28 : 크래프톤 아크 시스템 웍스 미스트 오버 아시아 서비스 계약 
doc29 : 뚜 뚜 블랙 핑크 지난 월 일 현지 시간 
doc30 : 농구 국가대표팀 존스 컵 이란 연승 행진 
doc31 : 포토 수영 
doc32 : 아시아

doc1758 : 페이스북 인공 지능 사용 태국 지도 작업 
doc1759 : 물 핑거루트 감량 미스코리아 다이어트 비법 공개 
doc1760 : 사자 판타지아 국제 영화제 초청 해외 개국 시기 개봉 
doc1761 : 작업복 외길 년 토피아 티뷰 통해 꼭 곳 반드시 
doc1762 : 블랙 핑크 방콕 끝 월드 투어 천사 
doc1763 : 제시카 발리 표정 부자 근황 공개 
doc1764 : 로드 블루문 펀드 동남아시아 사업 확대 추진 
doc1765 : 이상아 올해 나이 번의 이혼 사유 딸 윤서진 인스타 수입 만원 
doc1766 : 방탄소년단 정국 연예 매체 선정 매력 턱선 남자 스타 
doc1767 : 패스 호주 달러 등 개국 통화 환전 추가 
doc1768 : 자연 지애 배 농축 핑거 루트 추출 분말 
doc1769 : 동네 개구쟁이 한국 국가대표 
doc1770 : 공격 준비 
doc1771 : 무슬림 할랄 국내 약사 
doc1772 : 콘 진원 인도네시아 스타트업 멘토링 프로그램 개최 
doc1773 : 글로벌 루키 걸그룹 랜드 출격 확정 
doc1774 : 즙 핑거 루트 환 스틱 출시 
doc1775 : 청년 기획 세계 청년 축제 수영 대회 광주 개막 
doc1776 : 이용섭 시장 세계 인권 도시 포럼 추진 위서 광주 인권 도시 모델 김 당부 
doc1777 : 애완동물 발리 아기 오랑우탄 약 뒤 밀 반출 시도 러시 
doc1778 : 선박 안전 운항 세미나 콘퍼런스 개최 
doc1779 : 팝업 투어 인천공항 터미널 국제 면허증 발급 
doc1780 : 음악평론가 사이 호평 
doc1781 : 빈탄 청정 바다 패 보드 타 
doc1782 : 포토 수영 반칙 
doc1783 : 회 한국 인도네시아 기술 교류 상담 회 개최 
doc1784 : 인도네시아 불 수출 상담 등 쾌거 
doc1785 : 멸종 위해 인니 코모 섬 년 폐쇄 
doc1786 : 텐센트 전략 오토 배 모바일 게임 체스 러쉬 글로벌 스타 린지 개최 
doc1787 : 웅진 코웨이

In [28]:
tfidf_vectorizer=TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)

document_distances=(tfidf_matrix*tfidf_matrix.T)

print('유사도 분석을 위한' + str(document_distances.get_shape()[0])+'x'+str(document_distances.get_shape()[1])+'matrix를 만들었습니다')
print(document_distances.toarray())     

유사도 분석을 위한2298x2298matrix를 만들었습니다
[[1.         0.         0.         ... 0.         0.03479869 0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.03479869 0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [27]:
tfidf_vectorizer=TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)

document_distances=(tfidf_matrix*tfidf_matrix.T)

print()
print(document_distances.toarray())     

[[1.         0.         0.         ... 0.         0.03479869 0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.03479869 0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [ ]:
# 유사도 0.6 이상 조건 넣기 